In [ ]:
%pip install clickhouse-driver clickhouse-connect pandas seaborn matplotlib numpy python-dotenv
%pip install --upgrade pip
%pip install --upgrade lz4

In [ ]:
%load_ext dotenv
%dotenv

In [131]:
from clickhouse_driver import Client

In [ ]:
try:
    client = Client("localhost")
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

In [133]:
import clickhouse_connect as cc
import os

client = cc.get_client(host="localhost", port=8123, username="default", password="")

myorigin_dsn = os.getenv("DSN")

In [ ]:
try:
    client.query("SELECT 1")
    print("Connection successful")

    client.command("DROP TABLE IF EXISTS employee")

    list_columns_sql = f"""
    SELECT TOP 1 * FROM odbc ('{myorigin_dsn}', 'employee')
    """

    data = client.query(list_columns_sql)

    columns = data.column_names
    column_types = data.column_types

    # Create Column List
    column_list = []
    # <clickhouse_connect.datatypes.string.String object at 0x114bc5220>
    for column, type in zip(columns, column_types):
        column_list.append(f"{column} {type.name}")

    create_table_sql = f"CREATE TABLE employee ({','.join(column_list)}) ENGINE = MergeTree PRIMARY KEY (id, employeeId)"

    # client.command("DROP TABLE IF EXISTS employee")
    client.command(create_table_sql)

    # Create Data
    data = client.query(
        f"SELECT * FROM odbc ('{myorigin_dsn}', 'employee')"
    ).result_rows
    client.insert("employee", data, columns)

except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# ใช้ query เดิม
query = """
SELECT 
    toMonth(toDate(birthdate)) AS birth_month,
    toDayOfMonth(toDate(birthdate)) AS birth_day,
    count() AS count,
    groupArray(birthdate) AS birthdates
FROM employee
GROUP BY 
    toMonth(toDate(birthdate)),
    toDayOfMonth(toDate(birthdate))
HAVING count > 1
ORDER BY 
    birth_month,
    birth_day
"""

data = client.query(query)

# สร้าง DataFrame
column_names = ["birth_month", "birth_day", "count", "birthdates"]
df = pd.DataFrame(data.result_set, columns=column_names)

# สร้างกราฟความถี่ของวันเกิด
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x="birth_day", weights="count", bins=31, kde=True)
plt.xlabel("Birth Day")
plt.ylabel("Frequency")
plt.title("Frequency of Birth Days")
plt.show()

In [ ]:
# สมมติว่า data และ client ถูกกำหนดไว้แล้ว
data = client.query(
    "SELECT toMonth(toDate(birthdate)) AS birth_month, count() AS count FROM employee WHERE birthdate IS NOT NULL GROUP BY toMonth(toDate(birthdate)) ORDER BY birth_month ASC"
)
df = pd.DataFrame(data.result_set, columns=["birth_month", "count"])

# กำหนดชื่อเดือนภาษาอังกฤษ
thai_months = [
    "January",
    "February",
    "March",
    "April",
    "May",
    "June",
    "July",
    "August",
    "September",
    "October",
    "November",
    "December",
]

# สร้าง dictionary เพื่อแมปเลขเดือนกับชื่อเดือนภาษาอังกฤษ
month_dict = {i: month for i, month in enumerate(thai_months, 1)}

# แปลงเลขเดือนเป็นชื่อเดือนภาษาไทย
df["thai_month"] = df["birth_month"].map(month_dict)

plt.figure(figsize=(15, 6))
sns.barplot(
    x="thai_month",
    y="count",
    data=df,
    palette="viridis",
    order=[month_dict[i] for i in range(1, 13)],
    hue="thai_month",
    legend=False,
)

plt.xlabel("Birth Month", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.title("Frequency of Birth Months", fontsize=14)

# หมุนป้ายชื่อแกน x เพื่อให้อ่านง่ายขึ้น
plt.xticks(rotation=45, ha="right")

# ปรับ layout เพื่อให้แน่ใจว่าป้ายชื่อไม่ถูกตัด
plt.tight_layout()

plt.show()

In [ ]:
# ดึงข้อมูลสถานภาพสมรสจากฐานข้อมูล
data = client.query(
    "SELECT marital, count() AS count FROM employee GROUP BY marital ORDER BY count DESC"
)
# สร้าง DataFrame จากข้อมูลที่ได้
df = pd.DataFrame(data.result_set, columns=["marital", "count"])

# สร้างกราฟแท่งแสดงความถี่ของสถานภาพสมรส
plt.figure(figsize=(12, 6))
sns.barplot(x="marital", y="count", data=df, palette="viridis", hue="marital")
plt.xlabel("Marital Status")
plt.ylabel("Frequency")
plt.title("Frequency of Marital Status")

# ดึงข้อมูลเพศจากฐานข้อมูล
data = client.query(
    "SELECT gender, count() AS count FROM employee GROUP BY gender ORDER BY count DESC"
)
# สร้าง DataFrame จากข้อมูลที่ได้
df = pd.DataFrame(data.result_set, columns=["gender", "count"])

# สร้างกราฟแท่งแสดงความถี่ของเพศ
plt.figure(figsize=(12, 6))
sns.barplot(x="gender", y="count", data=df, palette="viridis", hue="gender")
plt.xlabel("Gender")
plt.ylabel("Frequency")
plt.title("Frequency of Gender")

# แสดงกราฟทั้งหมด
plt.show()

In [ ]:
# ดึงข้อมูลตัวอย่าง 1 แถวจากตาราง position ในฐานข้อมูล MSSQL ผ่าน ODBC
positions = client.query("SELECT TOP 1 * FROM odbc('{myorigin_dsn}', 'position')")

# ลบตาราง position ที่มีอยู่ (ถ้ามี) ใน ClickHouse
client.command("DROP TABLE IF EXISTS position")

# ดึงชื่อคอลัมน์และประเภทข้อมูลจากผลลัพธ์
columns = positions.column_names
column_types = positions.column_types

# สร้างรายการคอลัมน์พร้อมประเภทข้อมูลสำหรับใช้ในการสร้างตาราง
column_list = []
for column, type in zip(columns, column_types):
    column_list.append(f"{column} {type.name}")

# สร้าง SQL สำหรับการสร้างตาราง position ใน ClickHouse
create_table_sql = f"CREATE TABLE position ({','.join(column_list)}) ENGINE = MergeTree PRIMARY KEY (id)"

# สร้างตาราง position ใน ClickHouse
client.command(create_table_sql)

# ดึงข้อมูลทั้งหมดจากตาราง position ในฐานข้อมูล MSSQL
data = client.query(f"SELECT * FROM odbc ('{myorigin_dsn}', 'position')").result_rows

# นำเข้าข้อมูลจาก MSSQL ไปยังตาราง position ใน ClickHouse
client.insert("position", data, columns)

In [ ]:
query_sql = """
SELECT
    employee.firstnameEng || ' ' || employee.lastnameEng AS fullname,
    position.positionEng AS position,
    position.band AS band
FROM
    employee
    LEFT JOIN position ON employee.positionId = position.id
WHERE
    position IS NOT NULL
"""

employees_position = client.query(query_sql)

# สร้าง DataFrame โดยใช้ชื่อคอลัมน์ที่ถูกต้อง
df = pd.DataFrame(
    employees_position.result_rows, columns=["fullname", "position", "band"]
)

# กรอง band 0 และ '-' ออก
df = df[~df["band"].isin(["0", "-"])]

# แปลง band เป็นตัวเลขจำนวนเต็ม
df["band"] = pd.to_numeric(df["band"], errors="coerce").astype("Int64")

# นับความถี่ของแต่ละ band และกรองเฉพาะที่มีจำนวนมากกว่าหรือเท่ากับ 5
df_count = df["band"].value_counts().reset_index()
df_count.columns = ["band", "count"]
df_count = df_count[df_count["count"] >= 5].sort_values(
    "band"
)  # เรียงลำดับ band จากน้อยไปมาก

plt.figure(figsize=(12, 6))
sns.barplot(
    x="band", y="count", data=df_count, palette="viridis", hue="band", legend=False
)
plt.xlabel("Band")
plt.ylabel("Frequency")
plt.title("Frequency of Band (5 or more employees, excluding band 0 and -)")
plt.xticks(
    range(len(df_count)), df_count["band"].astype(int)
)  # แสดงค่า band เป็นจำนวนเต็ม
plt.tight_layout()  # ปรับ layout เพื่อให้แน่ใจว่าป้ายชื่อไม่ถูกตัด

# เพิ่มจำนวนคนบนแต่ละแท่ง
for i, v in enumerate(df_count["count"]):
    plt.text(i, v, str(v), ha="center", va="bottom")

plt.show()

In [ ]:
# สร้าง query SQL โดยใช้ f-string เพื่อแทนที่ค่า myorigin_dsn
query_sql = f"""
SELECT
    employee.firstnameEng || ' ' || employee.lastnameEng AS fullname,
    position.positionEng AS position_name,
    company.nameEng AS company_name,
    businessUnit.nameEng AS business_unit_name,
    division.nameEng AS division_name,
    department.nameEng AS department_name,
    section.nameEng AS section_name
FROM
    odbc ('{myorigin_dsn}', 'employee') AS employee
    LEFT JOIN odbc('{myorigin_dsn}', 'position') AS position ON position.id = employee.positionId
    LEFT JOIN odbc('{myorigin_dsn}', 'company') AS company ON company.id = employee.companyId
    LEFT JOIN odbc('{myorigin_dsn}', 'businessUnit') AS businessUnit ON businessUnit.id = employee.businessUnitId
    LEFT JOIN odbc('{myorigin_dsn}', 'division') AS division ON division.id = employee.divisionId
    LEFT JOIN odbc('{myorigin_dsn}', 'department') AS department ON department.id = employee.departmentId
    LEFT JOIN odbc('{myorigin_dsn}', 'section') AS section ON section.id = employee.sectionId
WHERE
    position.positionEng IS NOT NULL
"""

# ดึงข้อมูลจาก ClickHouse และสร้าง DataFrame
data = client.query(query_sql)
df = pd.DataFrame(
    data.result_rows,
    columns=[
        "fullname",
        "position_name",
        "company_name",
        "business_unit_name",
        "division_name",
        "department_name",
        "section_name",
    ],
)

# นับจำนวนพนักงานในแต่ละตำแหน่ง
position_counts = df["position_name"].value_counts().reset_index()
position_counts.columns = ["position_name", "count"]

# เลือกเฉพาะ Top 10 ตำแหน่งที่มีจำนวนพนักงานมากที่สุด
top_10_positions = position_counts.head(10)

# สร้างกราฟ Top 10 ตำแหน่งงานที่มีจำนวนพนักงานมากที่สุด
plt.figure(figsize=(12, 6))
sns.barplot(
    x="position_name",
    y="count",
    data=top_10_positions,
    palette="viridis",
    hue="position_name",
    legend=False,
)
plt.xlabel("Position")
plt.ylabel("Number of Employees")
plt.title("Top 10 Positions by Number of Employees")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

# เพิ่มจำนวนพนักงานบนแต่ละแท่ง
for i, v in enumerate(top_10_positions["count"]):
    plt.text(i, v, str(v), ha="center", va="bottom")

plt.show()